In [14]:
import numpy as np
import pandas as pd
# !pip install --upgrade pytorch_lightning wandb
from pytorch_lightning.core.lightning import LightningModule
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import WandbLogger

from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import RobustScaler, LabelEncoder, OrdinalEncoder, MinMaxScaler
from sklearn.pipeline import Pipeline
import numpy as np

from pathlib import Path
from argparse import ArgumentParser
import math
import wandb

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import Dataset, DataLoader
import pickle


data_dir = Path.home()/'data/kaggle/m5-forecasting-accuracy'

x_cat_cols = ['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id',
        'weekday', 'wday', 'month', 'year',
       'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2',
       'snap_CA', 'snap_TX', 'snap_WI']

x_cont_cols = ['sell_price']
num_train_val_days = 1913
num_test1_days = 28
num_test2_days = 28

src_len = 28
tgt_len = num_test1_days

#### TODO
 - normalize y
 - sales price is 0. fix it. 

In [ ]:
!ls $data_dir

#### Sales

In [ ]:
%%time
sales = pd.read_csv(data_dir/'sales_train_validation.csv')
print(f'sales.shape: {sales.shape}')
cat_cols = ['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']

# encode cat cols
encoders = {}
for col in cat_cols:
    encoder =  OrdinalEncoder()
    sales[[col]] = encoder.fit_transform(sales[[col]])
    sales[col] = sales[col].astype(np.long)
    encoders[col] = encoder
    
# change day column names to just day number
train_day_cols = {col: col.split('_')[1] for col in sales.columns if col.startswith('d_')}
sales.rename(columns=train_day_cols, inplace=True)

#### Add test data

In [ ]:
test_day_cols = [str(num_train_val_days + 1 + o) for o in range(56)]
for col in test_day_cols:
    sales[col] = 0
print(sales.shape)

In [ ]:
sales.tail(2)

In [ ]:
sample = pd.read_csv(data_dir/'sample_submission.csv')
sample.tail(2)

In [ ]:
num_days = len(train_day_cols) + len(test_day_cols)
num_stores = sales['store_id'].nunique()
num_items = sales['item_id'].nunique()
print('total days : ', num_days)
print('num store_items - ', num_stores * num_items)

In [ ]:
sales['item_id'].nunique()

#### Calendar

In [ ]:
calendar = pd.read_csv(data_dir/'calendar.csv')\
            .rename(columns={'d':'day'})

cat_cal_cols = ['wm_yr_wk', 'weekday', 'wday', 'month', 'year',
       'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2',
       'snap_CA', 'snap_TX', 'snap_WI']
# ignore_cal_cols = ['wm_yr_wk']

for col in cat_cal_cols:
    
    # impute
    if str(calendar[col].dtype)[:3] == 'obj':
        fill_value = 'abcxyz' 
    elif str(calendar[col].dtype)[:3] == 'int':
        fill_value = -1
    calendar[[col]] = SimpleImputer(strategy='constant', fill_value=fill_value).fit_transform(calendar[[col]])
    
    # encode
    if col not in encoders:
        encoders[col] = OrdinalEncoder().fit(calendar[[col]])
    calendar[[col]] = encoders[col].transform(calendar[[col]])
    calendar[col] = calendar[col].astype(np.long)
    
# change day column names to just day number
calendar['day'] = calendar['day'].apply(lambda x: x.split('_')[1])
calendar['day'] = calendar['day'].astype(np.long)

calendar.tail(2)

#### Prices

In [ ]:
%%time
prices = pd.read_csv(data_dir/'sell_prices.csv')
for col in ['store_id', 'item_id', 'wm_yr_wk']:
    prices[[col]] = encoders[col].transform(prices[[col]])
    prices[col] = prices[col].astype(np.long)

In [ ]:
prices.sort_values('wm_yr_wk',ascending=False).head(2)

### Merge

In [ ]:
%%time
sales2 = pd.melt(sales, id_vars=['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], 
                                       var_name='day', value_name='demand')
sales2['day'] = sales2['day'].astype(np.long)

sales2.sort_values('day', inplace=True)
calendar.sort_values('day', inplace=True)

sales2 = sales2.merge(calendar, on='day', how='left')
sales2 = sales2.merge(prices, on=['store_id', 'item_id', 'wm_yr_wk'], how='left')
sales2['sell_price'] = sales2['sell_price'].astype(np.float32)
sales2['sell_price'] = sales2['sell_price'].fillna(0.0)

sales2.sort_values(['item_id', 'store_id','day'], inplace=True)

# scale continuous columns
scalers = {}
for col in ['sell_price','demand']:
    scaler = MinMaxScaler()
    sales2[[col]] = scaler.fit_transform(sales2[[col]])
    scalers[col] = scaler

In [ ]:
sales2.to_parquet('combined.pq')
with open('encoders.pkl','wb') as f:
    pickle.dump(encoders,f)
    
with open('scalers.pkl','wb') as f:
    pickle.dump(scalers, f)

### Creating tensors

In [ ]:
%%time
sales2 = pd.read_parquet('combined.pq')
print(sales2.shape)
sales2.columns

In [ ]:
%%time
x = torch.tensor(sales2[x_cat_cols + x_cont_cols].values)
y = torch.tensor(sales2['demand'].values)

In [ ]:
sales2[x_cat_cols + x_cont_cols].dtypes

In [ ]:
%%time

# from fastai v2
def get_emb_size(nunique):
    return min(600, round(1.6 * nunique**0.56))

emb_sizes = [(sales2[col].nunique(), get_emb_size(sales2[col].nunique())) for col in x_cat_cols]

In [ ]:
with open('emb_sz.pkl','wb') as f:
    pickle.dump(emb_sizes,f )

In [ ]:
# group_size = num_items * num_stores
# group_size

In [ ]:
%%time
num_features = x.size(1)
x1 = x.view(-1, num_days, num_features).refine_names('item_store', 'day','features')\
        .align_to('day','item_store','features').contiguous()

y1 = y.view(-1, num_days).refine_names('item_store', 'day')\
    .align_to('day', 'item_store').contiguous()

print(f'x1.shape - {x1.shape} y1.shape - {y1.shape}')

In [ ]:
%%time
torch.save(x1.rename(None), 'x.pt')
torch.save(y1.rename(None), 'y.pt')


### Training

In [2]:
class M5DataSet(Dataset):
    def __init__(self,x, y, src_len, tgt_len, bsz, dstype='train'):
        assert dstype in ['train', 'test1', 'test2', 'val']
        self.x = x
        self.y = y
        self.src_len = src_len
        self.tgt_len = tgt_len
        self.bsz = bsz
        self.dstype = dstype
        
        self.val_days = self.src_len + self.tgt_len
        self.test_days = self.tgt_len * 2
        self.val_idx = self.x.size(0) - (self.val_days + self.test_days)
        self.test1_idx = self.x.size(0) - (self.src_len + self.test_days)
        print(f'val index - {self.val_idx}. test1_idx - {self.test1_idx}', )
        
    def __len__(self):
        if self.dstype == 'train':
            l = (self.x.size(0) - (self.src_len + self.val_days + self.test_days)) 
            return l
        
        if self.dstype =='val':
             return 1
        
        if self.dstype == 'test1':
            return 1
        
        return l
    
    def __getitem__(self, idx):
        if self.dstype == 'train':
            # we have 30490 item_stores. We may not be able to load them all. So randomly pick bsz items. 
            item_store_mask = list(np.random.randint(0, self.x.size(1),(self.bsz,)))
        elif self.dstype == 'val':
            item_store_mask = list(np.random.randint(0, self.x.size(1),(self.bsz,)))
            idx = self.val_idx
        elif self.dstype == 'test1':
            item_store_mask = list(np.arange(self.x.size(1)))
            idx = self.test1_idx
            print('test1 index - ', idx)
        
        x_src = self.x.rename(None)[idx:idx+self.src_len, item_store_mask, :]
        x_tgt = self.x.rename(None)[idx+self.src_len:idx+self.src_len+self.tgt_len, item_store_mask, :]
        y_src = self.y.rename(None)[idx:idx+self.src_len, item_store_mask]
        y_tgt = self.y.rename(None)[idx+self.src_len:idx+self.src_len+self.tgt_len, item_store_mask]
#         print(f'x.shape - {self.x.shape} y.shape - {self.y.shape} idx - {idx}. x_item.shape - {x_item.shape} y_item.shape - {y_item.shape}')
        return x_src, x_tgt, y_src, y_tgt, item_store_mask

# train_ds = M5DataSet(x1, y1, src_len, tgt_len, 200)
# train_dl = DataLoader(train_ds, batch_size=1, shuffle=True, pin_memory=True)

In [3]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)
    
# def insert_embedding(inp, dim, index, emb):
#     """
#     Replace columns with their embeddings. Works only with 2-d tensors.
#     TODO - make it work for multi-dim tensors

#     :param inp: tensor of two or more dimensions
#     :param dim: dimension along which tensor should be expanded by inserting the embedding
#     :param i: index of tensor along dim which is to be embedded
#     :param emb: Embedding of shape [v,d], where v vocab_size and d is embedding dimension
#     :return: 
#     """
#     # create a slice of the data to be replaced with embedding. 
#     s = inp.index_select(dim, torch.tensor([index])).squeeze(dim)
#     embedded = emb(s.type(torch.long))
    
#     first_indices = torch.arange(0,index)
#     last_indices = torch.arange(index+1,inp.size(dim))

#     return torch.cat([inp.index_select(dim, first_indices), embedded.type(inp.dtype), inp.index_select(dim, last_indices)], axis=dim)

In [4]:
%%time
gx = torch.load('x.pt')
gy = torch.load('y.pt')
print(f'gx.shape - {gx.shape} gy.shape - {gy.shape}')

gx.shape - torch.Size([1969, 30490, 17]) gy.shape - torch.Size([1969, 30490])
CPU times: user 8.16 ms, sys: 6.72 s, total: 6.73 s
Wall time: 6.71 s


In [46]:
class SalesModel(LightningModule):
    def __init__(self, hparams):
        super(SalesModel, self).__init__()
        self.hparams = hparams
        self.x_cat_cols = x_cat_cols
        self.x_cont_cols = x_cont_cols
        self.pos_encoder = PositionalEncoding(hparams.ninp, hparams.dropout)
        
        encoder_layers = nn.TransformerEncoderLayer(hparams.ninp, hparams.nhead, hparams.nhid, hparams.dropout)
        decoder_layers = nn.TransformerDecoderLayer(hparams.ninp, hparams.nhead, hparams.nhid, hparams.dropout)
        self.encoder = nn.TransformerEncoder(encoder_layers, hparams.nlayers)
        self.decoder = nn.TransformerDecoder(decoder_layers, hparams.nlayers)
        
#         self.lin = nn.Linear()
#         self.transformer = nn.Transformer(d_model=hparams.ninp, nhead=hparams.nhead, 
#                                           num_encoder_layers=hparams.nlayers,
#                                           num_decoder_layers=hparams.nlayers,
#                                           dim_feedforward=hparams.nhid)
        self.transformer = nn.Transformer(d_model=hparams.ninp,
                                          custom_encoder=self.encoder, 
                                          custom_decoder = self.decoder)
        self.criterion = nn.MSELoss()
        self.lin1 = nn.Linear(hparams.ninp, 50)
        self.lin2 = nn.Linear(50, 1)
        self.sigmoid = nn.Sigmoid()
        self.relu = nn.ReLU()
        
        print('reading data', flush=True)
        self.x = gx
        self.y = gy

        with open('emb_sz.pkl','rb') as f:
            emb_szs = pickle.load(f)
        print(f'emb_szs - {emb_szs}')
                    
        self.embs = nn.ModuleList([nn.Embedding(e[0],e[1]) for e in emb_szs])
#         self.init_weights()
        
    def init_weights(self):
        initrange = 0.1
#         self.src_embedding.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)
        
    @staticmethod
    def add_model_specifi_args(parent_parser):
        parser = ArgumentParser(parents=[parent_parser], add_help=False)
        parser.add_argument('--bsz', default=20, type=int, help='batch_size', )
        parser.add_argument('--src-len', default=90, type=int, help='source length')
        parser.add_argument('--tgt-len', default=28, type=int, help='target length')
        parser.add_argument('--ninp', default=320, type=int, help='expected features in the input')
        parser.add_argument('--nhead', default=4, type=int, help='number of attention heads')
        parser.add_argument('--nhid', default=256, type=int, help='dimesion of feed-forward network model')
        parser.add_argument('--nlayers', default=2, type=int, help='number of encoder layers')
        parser.add_argument('--dropout', default=0.2, type=float, help='dropout')
        
        # they are not hyper params, but adding them as pytorch lightening can save them
        parser.add_argument('--num-cat-cols', default=len(x_cat_cols), type=int, help='number of categorical columns')
        parser.add_argument('--num-cont-cols', default=len(x_cont_cols), type=int, help='number of numeric columns')
        return parser
    
#     def _generate_square_subsequent_mask(self, sz):
#         # populate the lower triangle with True and rest with False
#         return torch.tril(torch.ones(sz, sz)) == 1.0
    
    def prepare_data(self):
        pass
        
    def train_dataloader(self):
        train_ds = M5DataSet(self.x, self.y, self.hparams.src_len, self.hparams.tgt_len, self.hparams.bsz,dstype='train')  
        print(f'train_ds.length - {len(train_ds)}')
        train_dl = DataLoader(train_ds, batch_size=1, shuffle=True, pin_memory=True)
        return train_dl
    
    def val_dataloader(self):
        train_ds = M5DataSet(self.x, self.y, self.hparams.src_len, self.hparams.tgt_len, self.hparams.bsz,dstype='val')  
        print(f'val.length - {len(train_ds)}')
        train_dl = DataLoader(train_ds, batch_size=1, shuffle=False, pin_memory=True)
        return train_dl

    
    def emb_lookups(self, xb, yb=None):
        embs_t = []
        for idx in range(self.hparams.num_cat_cols):
#             print('looking up for ', idx)
            embs_t.append(self.embs[idx](xb[:,:,idx].type(torch.long)))
        xb_cat = torch.cat(embs_t, dim=2)
        xb_cont = xb[:,:,self.hparams.num_cat_cols:]
        
        if yb is not None:
            xb = torch.cat([xb_cat, xb_cont.type(xb_cat.dtype), yb.unsqueeze(2).type(xb_cat.dtype)], dim=2)
        else:
            xb = torch.cat([xb_cat, xb_cont.type(xb_cat.dtype)], dim=2)
            
        #pad to adjust the feature dimension
        dim3_shortfall = self.hparams.ninp - xb.size(2)
        assert dim3_shortfall >= 0
        pad = nn.ConstantPad1d(padding=(0,dim3_shortfall),value=0)
        xb = pad(xb) 

        return xb

    def forward(self, x_src, y_src, x_tgt):        
        x_src = self.emb_lookups(x_src, y_src)
        x_tgt = self.emb_lookups(x_tgt)
            
        x_src = self.pos_encoder(x_src)
#         print('shape after pos encoder - ', x_src.size())
        out = self.transformer(x_src, x_tgt)
#         print('shape after transformer - ', out.size())
        out = self.relu(self.lin1(out))
        out = self.sigmoid(self.lin2(out))
        
        return out
    
    def compute_loss(self, batch, batch_idx):
        x_src, x_tgt, y_src, y_tgt, item_store_mask = batch
        x_src = x_src.squeeze(0)
        x_tgt = x_tgt.squeeze(0)
        y_src = y_src.squeeze(0)
        y_tgt = y_tgt.squeeze(0)
        
        yhat_tgt = self(x_src, y_src, x_tgt)
        loss = self.criterion((yhat_tgt).reshape(-1).type(torch.float32), (y_tgt).reshape(-1).type(torch.float32))
        return loss, y_tgt, yhat_tgt
        
    def training_step(self, batch, batch_idx):
        loss, y_tgt, yhat_tgt = self.compute_loss(batch, batch_idx)
        metrics = {'loss': loss, 'yhat_tgt_train_sum':yhat_tgt.sum().item()}

        if batch_idx%10 == 0:
#             print(f'{batch_idx} train loss: {loss}  yhat_tgt.sum: {yhat_tgt.sum().item()}  y_tgt.sum: {y_tgt.sum().item()}')
#             wandb.log(metrics)   
            self.logger.log_metrics(metrics)
        return metrics    
    
    def validation_step(self, batch, batch_idx):
        loss, y_tgt, yhat_tgt = self.compute_loss(batch, batch_idx)
        metrics = {'val_loss': loss,'yhat_tgt_val_sum':yhat_tgt.sum().item()}  
        if batch_idx%10 == 0:
#             print(f'{batch_idx} val loss: {loss}  yhat_tgt.sum: {yhat_tgt.sum().item()}  y_tgt.sum: {y_tgt.sum().item()}')
#             wandb.log(metrics)   
            self.logger.log_metrics(metrics)
        return metrics
        
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=0.0001)
        schedulers = [{
             'scheduler': ReduceLROnPlateau(optimizer,patience=10, verbose=True),
             'monitor': 'loss', # Default: val_loss
             'interval': 'step',
             'frequency': 1
          }]
        scheduler = ReduceLROnPlateau(optimizer,)
#         scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, 
#                                                         max_lr=0.01, steps_per_epoch=len(data_loader), epochs=10)
        return optimizer
    
#     def optimizer_step(self, current_epoch, batch_idx, optimizer, optimizer_idx,
#                        second_order_closure=None):
#         optimizer.step()
#         if batch_idx == 5:
#             for name, param in model.named_parameters():
#                 if param.requires_grad:
#                     pass
# #                     print(name, param.grad)
#         optimizer.zero_grad()
    
#     def test(self):
#         dl = self.test_dataloader()
#         batch = next(iter(dl))
#         return batch
    
    
#     def test_dataloader(self):
#         test_ds = M5DataSet(self.x, self.y, self.hparams.src_len, self.hparams.tgt_len, self.hparams.bsz, dstype='test1')  
#         test_dl = DataLoader(test_ds, batch_size=1, shuffle=False)
#         return test_dl
        

In [48]:
# bsz = 200
# model = SalesModel(hparams)

parser = ArgumentParser()
parser = SalesModel.add_model_specifi_args(parser)
hparams = parser.parse_args('--bsz 1000 --ninp 320 --nhid 512 --nlayers 1'.split())

checkpoint_callback = ModelCheckpoint(
    filepath='models/weights.ckpt',
    verbose=True
)

model = SalesModel(hparams)

wandb_logger = WandbLogger(name='achinta',project='kaggle-m5-forecasting-accuracy')
# run validation every 10 steps
trainer = Trainer(gpus=1,max_epochs=1,auto_lr_find=False, val_check_interval=10,logger=wandb_logger)
trainer.fit(model)
trainer.save_checkpoint('models/weights_v1.1.ckpt')

reading data


GPU available: True, used: True
No environment variable for node rank defined. Set as 0.
CUDA_VISIBLE_DEVICES: [0]


emb_szs - [(3049, 143), (7, 5), (3, 3), (10, 6), (3, 3), (7, 5), (7, 5), (12, 6), (6, 4), (31, 11), (5, 4), (5, 4), (3, 3), (2, 2), (2, 2), (2, 2)]


In [42]:
trainer.s



### testing

In [ ]:
%%time
model = SalesModel.load_from_checkpoint('models/weights.ckpt')
x_src, x_tgt, y_src, y_tgt, item_store_mask = model.test()
x_src = x_src.squeeze(0)
x_tgt = x_tgt.squeeze(0)
y_src = y_src.squeeze(0)
y_tgt = y_tgt.squeeze(0)
print(f'x_src.shape - {x_src.shape}, x_tgt.shape - {x_tgt.shape} , y_src.shape - {y_src.shape} , y_tgt.shape - {y_tgt.shape}')

model.eval()
print('starting inference...')
yhat_tgt = model(x_src, y_src, x_tgt)
yhat_tgt.shape

In [ ]:
yhat_tgt = yhat_tgt.refine_names('days','item_store','demand')
yhat_tgt_aligned = yhat_tgt.align_to('item_store','days','demand').squeeze(2).detach().numpy()
print(f'yhat.shape: ', yhat_tgt_aligned.shape)

# create preds df
preds = pd.DataFrame()
preds['id'] = sales['id']

# read scalers
with open('scalers.pkl','rb') as f:
    scalers = pickle.load(f)


pred_ids = preds['id'].tolist()
# eval df should also be submitted (days 1942 to 1969)
eval_ids = ['_'.join(o.split('_')[:5] + ['evaluation']) for o in pred_ids]
eval_df = pd.DataFrame({'id': eval_ids})

for idx in range(num_test1_days):
    preds['F' + str(idx+1)] = yhat_tgt_aligned[:,idx]
    preds['F' + str(idx+1)] = scalers['demand'].inverse_transform(preds[['F' + str(idx+1)]])
    
    eval_df['F' + str(idx+1)] = 0.0
    
out_df = pd.concat([preds,eval_df],axis=0)
print(out_df.shape)
preds.head()

In [ ]:
out_df.describe()

In [ ]:
out_df.to_csv('preds.csv', index=False)
!kaggle competitions submit -c m5-forecasting-accuracy -f preds.csv -m "transformers 3"

In [ ]:
# !head preds.csv

## Playground

In [ ]:
x_src.shape

In [ ]:
item_store_mask = list(np.random.randint(0, 10,3))
item_store_mask

In [ ]:
torch.randn(10).sum().item()

In [ ]:
sample.head(2)

In [ ]:
dir(hparams)`

In [ ]:
sub = pd.read_csv(data_dir/'sample_submission.csv')

In [ ]:

    
eval_df.head()

In [ ]:
sub.shape

In [ ]:
preds.shape

In [ ]:
preds.head()

In [ ]:
torch.cuda.is_available()